In [64]:
%matplotlib inline 
import pandas as pd
import numpy as np
import pylab as plt
import nltk
import re
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD

# Обработка писем
https://www.kaggle.com/antlarac/d/kaggle/hillary-clinton-emails/hillary-s-emails

# Для работы с SQL может пригодиться
https://github.com/catherinedevlin/ipython-sql

In [2]:
!ls "hillary-clinton-emails"

Aliases.csv        Emails.csv         database.sqlite
EmailReceivers.csv Persons.csv        hashes.txt


Извлекли все данные

In [3]:
emails = pd.read_csv('hillary-clinton-emails/Emails.csv')
persons = pd.read_csv('hillary-clinton-emails/Persons.csv')
emailReceivers = pd.read_csv('hillary-clinton-emails/EmailReceivers.csv')
aliases = pd.read_csv('hillary-clinton-emails/Aliases.csv')

## Датасет состоит из 4 файлов, 
#### В файле Persons находится информация о том, с кем происходило общение отправка/получение сообщений. 
#### В файле Alias предоставлена информация из файла persons, но обработанная: все слова приведены к нижнему регистру, добавлены персональные id номера, разные написания одного и того же адресата объединены одним id номером, поэтому записей в данном файле больше, чем в файле Persons. 
#### В файле EmailReceivers предоставлена информация о получателях сообщения в формате персонального id сообщения и id получателя из файла Aliases
#### В фвйле Emails предоставлена информация о сообщениях, нас интересует поле RawText, где находится сам текст письма.

## Работа с текстами писем

In [4]:
emails.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,...,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
3,4,C05739550,CAIRO CONDEMNATION - FINAL,H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739550...,F-2015-04841,...,NaN,"Mills, Cheryl D <MillsCD@state.gov>","Mitchell, Andrew B","Wednesday, September 12,2012 12:44 PM",F-2015-04841,C05739550,05/13/2015,RELEASE IN PART,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
4,5,C05739554,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739554...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739554,05/13/2015,RELEASE IN PART,"H <hrod17@clintonemail.com>\nFriday, March 11,...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...


Нас интересует столбец RawText, который содержит тексты писем.

In [5]:
emails.RawText[5]

"B6\nUNCLASSIFIED\nU.S. Department of State\nCase No. F-2015-04841\nDoc No. C05739559\nDate: 05/13/2015\nSTATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.\nSUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER.\nRELEASE IN\nPART B6\nFrom:\nSent\nTo:\nSubject\nPis print.\n-•-...-^\nH < hrod17@clintonernailcom>\nWednesday, September 12, 2012 2:11 PM\n°Russorv@state.gov'\nFw: Meet The Right-Wing Extremist Behind Anti-fvluslim Film That Sparked Deadly Riots\nFrom [meat)\nSent: Wednesday, September 12, 2012 01:00 PM\nTo: 11\nSubject: Meet The Right Wing Extremist Behind Anti-Muslim Film That Sparked Deadly Riots\nhtte/maxbiumenthal.com12012/09/meet-the-right-wing-extremist-behind-anti-musiim-tihn-that-sparked-\ndeadly-riots/\nSent from my Verizon Wireless 4G LTE DROID\nUNCLASSIFIED\nU.S. Department of State\nCase No. F-2015-04841\nDoc No. C05739559\nDate: 05/13/2015\nSTATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.\nSUBJECT TO AGREEMENT ON SENSITIVE INFORMATION

Разобьем тексты по слову UNCLASSIFIED, очень похоже, что оно разбивает письма между собой или их части.

In [6]:
x = emails.RawText[1].split("UNCLASSIFIED")

Посмотрим на длины отрывков, я считаю, что представленный ниже отрывок считается техническим и его не надо учитывать в нашей выборке, поэтому минимальную длину отрывка для обработки далее задам в 300 символов.

In [7]:
for i in x:
    print(len(i))
print(x[2])

0
1867
231
2438
231
3045
231

U.S. Department of State
Case No. F-2015-04841
Doc No. C05739546
Date: 05/13/2015
STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.
SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-SCB0045000



In [8]:
x[1]

'\nU.S. Department of State\nCase No. F-2015-04841\nDoc No. C05739546\nDate: 05/13/2015\nSTATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.\nSUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER.\nRELEASE IN\nPART B6\nFrom:\nSent:\nTo:\nSubject:\nAttachments:\nB6\nThursday, March 3, 2011 9:45 PM\nH: Latest How Syria is aiding Qaddafi and more... Sid\nhrc memo syria aiding libya 030311.docx; hrc memo syria aiding libya 030311.docx\nCONFIDENTIAL\nMarch 3, 2011\nFor: Hillary\nFrom: Sid\nRe: Syria aiding Qaddafi\nThis memo has two parts. Part one is the report that Syria is providing air support for Qaddafi. Part two is a note\nto Cody from Lord David Owen, former UK foreign secretary on his views of an increasingly complex crisis. It\nseems that the situation is developing into a protracted civil war with various nations backing opposing sides\nwith unforeseen consequences. Under these circumstances the crucial challenge is to deprive Qaddafi of his\nstrategic dept

## Предобработка текстов:

- Удалить слова, которые имеют отношение к техническим аспектам письма.
- удалить стоп-слова
- сделать лематизацию
- удалить разного рода символы
- разделить письма на тексты по слову UNCLASSIFIED
- отбросить тексты длиной меньше 250 символов
- сделать из всех текстов один большой набор и с ним уже работать

Добвил стоп-слова после нескольких прогонов на полном датасете и убрал из выборки слова меньше 3 символов


In [263]:
my_stop_words = ['original', 'message', 'release', 'part','united', 'state', 'don', 't', 'it', 'a', 's', 'http', 'www', 'i','m']

In [264]:
def clean_text(text):
    stop_words = stopwords.words("english")
    stop_words.extend(my_stop_words)
    lemmatizer = WordNetLemmatizer()

    text = text.replace("\n", " ")
    text = text.strip()
    text = (lemmatizer.lemmatize(word) for word in text.split())
    text = [word for word in text if not (word in stop_words)]

    return " ".join(word for word in text)

In [265]:
def replase_garbage(row):
    parse_dateMDET = r'\w+,\s\w+\s\d*,\s\d{4}\s\d:\d{2}\s\w+'
    parse_doc = r'\d*.docx'
    parse_USD = r'U.S. \w+ \w+ \w+'
    parseCaseNO = r'Case No. \w-\d{4}-\d{5}'
    parseDate = r'Date: \d{2}/\d{2}/\d{2}|\w+\s\d*,\s\d{4}'
    parseSys = r'From:*|Sent:*|To:*|Subject:*|Attachments:*|Re:*|RE:|For:*|Cc:*|Fax:|Tel:|Fw:|FW:'
    parseDoc_NO = r'Doc No.*|SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER.|SUBJECT TO AGEMENT ON SENSITIVE INFORMATION & DACTIONS. NO FOIA WAIVER.'
    parseDep = r'STATE DEPT.*'
    parseNum = r'\d*'
    parseStrDep = r'\w.\w.\s\w+\s\w+\s\w+\s\w+\s\w+.*\s\w-\d{4}-\d{5}\s\w+\s\w+.\s\w\d{8}\s\w+:\s\d{2}/\d{2}/\d{4}'
    parseEmail = r'<\w+@\w+.\w+>|\b\w+@\w+\b'
    parseLetters = r'\s\b.\s'
    
    rw = re.sub(parse_dateMDET, "", row)
    rw = re.sub(parseStrDep, "", rw)
    rw = re.sub(parse_doc, "", rw)
    rw = re.sub(parse_USD, "", rw)
    rw = re.sub(parseEmail, "", rw)
    rw = re.sub(parseDate, "", rw)
    rw = re.sub(parseSys, "", rw)
    rw = re.sub(parseDoc_NO, "", rw)
    rw = re.sub(parseDep, "", rw)
    rw = re.sub(parseNum, "", rw)
    rw = re.sub(parseCaseNO, "", rw)
    rw = re.sub(parseLetters, "", rw)
    
    rw = rw.replace("AM", " ")
    rw = rw.replace("PM", " ")
    rw = rw.replace(",", " ")
    rw = rw.replace(".", " ")
    rw = rw.replace("-", " ")
    rw = rw.replace(":", " ")
    rw = rw.replace("/", " ")
    rw = rw.replace("\\", " ")
    rw = rw.replace("<", " ")
    rw = rw.replace(">", " ")
    rw = rw.replace("(", " ")
    rw = rw.replace(")", " ")
    rw = rw.replace("•", " ")
    rw = rw.replace("^", " ")
    rw = rw.replace("°", " ")
    rw = rw.replace("!", " ")
    rw = rw.replace("?", " ")
    rw = rw.replace("[", " ")
    rw = rw.replace("]", " ")
    rw = rw.replace("don't", " ")
    rw = rw.replace("it's", " ")
    
    rw = re.sub(r" +", " ", rw)
    rw = rw.lower()
    return clean_text(rw)

2) Разделяем сообщения по слову UNCLASSIFIED

In [288]:
sa = []
for i in range(1000):
    sa.append(emails.RawText[i].split("UNCLASSIFIED"))

3) Отбрасываем все отрывки короче 250 символов и складываем все в один список

In [289]:
emTest = []
for i in range(1000):
    for fragment in sa[i]:
        if len(fragment) > 250:
            emTest.append(fragment)

In [290]:
words_for_bigr = " ".join(replase_garbage(fragment) for fragment in emTest).strip().split()
msg_temp = (replase_garbage(fragment) for fragment in emTest if fragment)
messages = np.array([message for message in msg_temp if len(message.split()) > 3])

In [291]:
messages.shape

(1708,)

Найдем наиболее часто встречающиеся биграммы в датасете

In [292]:
from itertools import islice, zip_longest
words = re.findall("\w+", str(words_for_bigr))
print(Counter(zip(words, islice(words, 1, None))).most_common(10))

[(('mill', 'cheryl'), 520), (('el', 'keib'), 400), (('sullivan', 'jacob'), 369), (('abedin', 'huma'), 337), (('secretary', 'clinton'), 303), (('prime', 'minister'), 241), (('i', 'm'), 237), (('sensitive', 'source'), 229), (('source', 'comment'), 218), (('full', 'gov'), 185)]


Выделим коллокации из двух слов по PMI с помощью nltk

In [293]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(words_for_bigr)

In [294]:
finder.nbest(bigram_measures.pmi, 10)

[('"actionable', 'intelligence"'),
 ('"advice', 'consent"'),
 ('"almost', 'unthinkable'),
 ('"brand"', 'intoreality'),
 ('"build', 'brick'),
 ('"bust', 'up"'),
 ('"buy', 'china"'),
 ('"call', 'out"'),
 ('"cheryl', 'mills"'),
 ('"chi', 'comms"')]

## Попробуем кластеризовать

In [295]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=500, min_df=10)
matrix = vectorizer.fit_transform(messages).toarray()

In [296]:
print(matrix.shape)

(1708, 2796)


Аггломеративная кластеризация (neighbour joining)

In [297]:
from sklearn.cluster.hierarchical import AgglomerativeClustering

model = AgglomerativeClustering(n_clusters=10,affinity='euclidean', linkage='complete')
preds = model.fit_predict(matrix)

In [298]:
print(list(preds))

[5, 1, 1, 1, 5, 2, 2, 1, 1, 2, 1, 2, 1, 6, 6, 1, 1, 2, 1, 2, 9, 9, 9, 0, 1, 1, 1, 5, 5, 0, 1, 1, 1, 5, 5, 9, 9, 9, 2, 2, 5, 5, 5, 5, 0, 1, 1, 1, 5, 0, 1, 1, 1, 1, 1, 1, 5, 3, 1, 1, 1, 1, 0, 2, 2, 5, 5, 5, 6, 5, 2, 2, 1, 1, 1, 1, 1, 1, 5, 9, 2, 2, 2, 2, 0, 1, 1, 0, 0, 1, 1, 0, 2, 3, 2, 0, 9, 9, 9, 9, 9, 9, 9, 9, 9, 5, 5, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 4, 9, 9, 3, 9, 9, 3, 0, 2, 2, 3, 2, 2, 8, 2, 2, 2, 8, 5, 2, 2, 8, 1, 1, 0, 1, 1, 0, 2, 2, 2, 8, 2, 9, 2, 2, 8, 0, 2, 5, 9, 1, 9, 2, 9, 9, 0, 0, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 1, 5, 5, 5, 1, 5, 5, 5, 1, 5, 5, 5, 5, 0, 2, 2, 1, 5, 5, 5, 1, 5, 5, 5, 2, 6, 6, 6, 1, 5, 5, 5, 1, 5, 5, 5, 1, 2, 2, 9, 1, 1, 1, 2, 2, 2, 9, 6, 2, 1, 1, 1, 1, 1, 1, 1, 5, 2, 9, 5, 2, 9, 5, 6, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 9, 5, 9, 5, 5, 5, 5, 0, 6, 5, 5, 2, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 9, 9, 9, 1, 1, 1, 1, 3, 5, 9, 1, 1, 1, 1, 5, 5, 9, 1, 1, 2, 2, 1, 0, 0, 0, 0, 

In [299]:
svd = TruncatedSVD(n_components=1000)
features = svd.fit_transform(matrix)

In [300]:
svd.explained_variance_ratio_.sum()

0.98429996425316024

KMeans
Тестировал на выборке в 1000 сообщений

In [37]:
from sklearn.cluster import KMeans

model = KMeans(n_clusters=5, random_state=1)
preds = model.fit_predict(matrix)
print(preds[:25])
print("=========================")
model = KMeans(n_clusters=10, random_state=1)
preds = model.fit_predict(matrix)
print(preds[:25])
print("=========================")
model = KMeans(n_clusters=15, random_state=1)
preds = model.fit_predict(matrix)
print(preds[:25])
print("=========================")
model = KMeans(n_clusters=20, random_state=1)
preds = model.fit_predict(matrix)
print(preds[:25])
print("=========================")
model = KMeans(n_clusters=25, random_state=1)
preds = model.fit_predict(matrix)
print(preds[:25])


[2 1 1 1 2 2 2 1 1 2 1 2 2 3 3 1 1 2 1 2 3 3 3 2 1]
[0 4 4 4 5 5 8 4 4 8 4 8 8 9 9 4 4 8 4 8 9 9 2 8 3]
[ 6 10 10 10  0  0  1 10 10  1 10  1  1  1  9 10 10  1 10  1  9  9  9  1 10]
[ 4 12 12 12  8  4  4 12 12  5 12 15  5  5  5 12 12  5 12 15  5  5  5  4 17]
[11 20 20 20  6  6  0 20 20  0 20  3  0 16 16 20 20  0 20  3 16 16 16  9  5]


10 кластеров показывают наиболее хороший результат распределения

In [301]:
Number_of_clusters = 10

In [302]:
model = KMeans(n_clusters=Number_of_clusters, random_state=1)
preds = model.fit_predict(matrix)
print(preds[:25])

[2 6 6 6 0 0 2 6 6 2 6 2 2 1 1 6 6 2 6 2 1 1 8 2 6]


In [303]:
def count_top_N_words_in_cluster(cluster,n):
    joined_cluster = " ".join(cluster)
    words = joined_cluster.split()
    resulting_count = Counter(words)
    top_10 = resulting_count.most_common(10)
    top_words = re.findall(r'\b[a-z]\w+\b', str(top_10))
    text_top = " ".join(word for word in top_words)
    
    return text_top

In [304]:
for i in range(Number_of_clusters):
    print("cluster size: ", messages[preds == i].shape[0])
    print(count_top_N_words_in_cluster(messages[preds == i], Number_of_clusters))
    print()

cluster size:  203
cheryl mill sullivan jacob gov secretary full

cluster size:  185
wa said attack libya benghazi security ha american

cluster size:  530
com gov sullivan pm amazon would wa ha

cluster size:  163
huma abedin gov sep full tue lona schedule valmoro

cluster size:  84
el source keib government minister militia al national ntc jalil

cluster size:  83
call huma abedin full gov talk may fm mon

cluster size:  165
source libyan magariaf libya government al qaddafi individual new security

cluster size:  36
hanley saturday monica pdb mailto dan king call today fm

cluster size:  243
clinton wa secretary question ha people one

cluster size:  16
fri aug sullivan ines philippe jacob gov aren



## Теперь сделаем все шаги для всей выборки

In [307]:
sa = []
for i in range(len(emails.RawText)):
    sa.append(emails.RawText[i].split("UNCLASSIFIED"))

In [308]:
emTest = []
for i in range(len(emails.RawText)):
    for fragment in sa[i]:
        if len(fragment) > 250:
            emTest.append(fragment)

In [309]:
words_for_bigr = " ".join(replase_garbage(fragment) for fragment in emTest).strip().split()
msg_temp = (replase_garbage(fragment) for fragment in emTest if fragment)
messages = np.array([message for message in msg_temp if len(message.split()) > 3])

In [310]:
messages.shape

(11577,)

Биграммы для всего датасета:

In [311]:
from itertools import islice, zip_longest
words = re.findall("\w+", str(words_for_bigr))
print(Counter(zip(words, islice(words, 1, None))).most_common(10))

[(('mill', 'cheryl'), 4268), (('abedin', 'huma'), 3752), (('sullivan', 'jacob'), 3153), (('i', 'm'), 1401), (('huma', 'abedin'), 1131), (('secretary', 's'), 1101), (('white', 'house'), 1060), (('prime', 'minister'), 1023), (('jiloty', 'lauren'), 921), (('secretary', 'clinton'), 859)]


коллокации из двух слов по PMI с помощью nltk

In [312]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(words_for_bigr)

In [313]:
finder.nbest(bigram_measures.pmi, 10)

[('""one', 'liners"'),
 ('"\'"', 'unregulated'),
 ('"\'hi', 'isf'),
 ('"absenting', 'himself"'),
 ('"actionable', 'intelligence"'),
 ('"active', 'robust"'),
 ('"activist', 'justice"'),
 ('"actors"', '—yet—demand'),
 ('"afterfew', 'cheeto'),
 ('"ahmed', 'amazed"')]

## Кластеризуем

In [314]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=500, min_df=10)
matrix = vectorizer.fit_transform(messages).toarray()

In [315]:
print(matrix.shape)

(11577, 9722)


# Аггломеративная кластеризация (neighbour joining)

In [168]:
from sklearn.cluster.hierarchical import AgglomerativeClustering

model = AgglomerativeClustering(n_clusters=10,affinity='euclidean', linkage='complete')
preds = model.fit_predict(matrix)

In [169]:
print(list(preds))

[8, 3, 3, 3, 1, 8, 8, 3, 3, 8, 3, 8, 0, 8, 8, 3, 3, 8, 3, 8, 1, 1, 0, 0, 1, 1, 1, 8, 8, 0, 1, 1, 1, 8, 8, 5, 5, 5, 8, 8, 5, 5, 8, 5, 0, 1, 1, 1, 8, 0, 1, 1, 1, 1, 1, 1, 8, 8, 1, 1, 1, 1, 0, 8, 8, 8, 8, 8, 1, 8, 8, 8, 8, 3, 8, 3, 3, 3, 8, 8, 1, 1, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 5, 1, 5, 8, 1, 1, 1, 1, 8, 0, 0, 0, 8, 8, 1, 1, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8, 3, 3, 1, 3, 3, 1, 1, 0, 0, 1, 0, 0, 8, 5, 0, 0, 0, 7, 0, 0, 0, 3, 3, 1, 3, 3, 1, 8, 0, 0, 0, 0, 3, 0, 0, 0, 8, 8, 7, 1, 1, 8, 1, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 8, 8, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 8, 8, 8, 8, 1, 0, 0, 0, 1, 0, 0, 0, 8, 0, 0, 8, 3, 1, 3, 8, 0, 0, 8, 8, 8, 3, 1, 3, 3, 1, 1, 3, 8, 8, 8, 8, 8, 8, 8, 1, 8, 8, 8, 8, 1, 1, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 1, 8, 8, 8, 1, 8, 8, 8, 8, 8, 8, 1, 1, 8, 8, 8, 8, 8, 8, 8, 8, 3, 3, 3, 3, 3, 3, 8, 8, 8, 3, 3, 3, 3, 8, 8, 8, 3, 3, 3, 3, 8, 8, 8, 3, 3, 7, 1, 0, 8, 8, 8, 8, 

In [170]:
svd = TruncatedSVD(n_components=1000)
features = svd.fit_transform(matrix)

In [171]:
svd.explained_variance_ratio_.sum()

0.64184436180689586

# Kmeans

In [316]:
Number_of_clusters = 10

In [317]:
model = KMeans(n_clusters=Number_of_clusters, random_state=1)
preds = model.fit_predict(matrix)
print(preds[:25])

[2 1 1 1 4 2 2 1 1 2 1 2 2 9 9 1 1 2 1 2 9 1 9 1 1]


In [318]:
for i in range(Number_of_clusters):
    print("cluster size: ", messages[preds == i].shape[0])
    print(count_top_N_words_in_cluster(messages[preds == i], Number_of_clusters))
    print()

cluster size:  296
sullivan philippe ines jacob muscatine monica lissa hanley speech huma

cluster size:  289
source libyan libya government al el magariaf minister individual national

cluster size:  5927
cheryl mill huma abedin call sullivan jacob full wa

cluster size:  244
lona valmoro huma abedin schedule secretary pm cheryl mill

cluster size:  231
anne slaughter marie sullivan mill cheryl jacob full abedin speech

cluster size:  284
lauren jiloty call huma abedin aug mashabane update list thu

cluster size:  187
pm office secretary room meeting depart arrive route department

cluster size:  227
call deal party dup gordon said agreement davutoglu text power

cluster size:  200
holbrooke call abedin huma eikenberry wa sullivan jacob full ha

cluster size:  3692
wa ha said would government one president american



Как видно из полученных результатов, часто встречаются слова, которые сложно отнести к теме, поэтому для дальнейшей оптимизации можно строить кластеризацию, анализировать топ 10 слов, если там будут слова, которые не имеют отношения к теме - удалять их. Это, я считаю, поможет улучшить кластеризацию. Я этого не проделал до варианта, который мне покажется подходящим из-за нехватки ресурсов вычислительной системы (вычисления происходят очень долго). 

Наверное, еще можно было бы убрать слова, которые короче 3 символов, потому что они не несут особой смысловой нагрузки. Например:
- wa
- ha
- thu
- al
- el

Возможно это тоже должно помочь.